In [1]:
import pennylane as qml
from pennylane import numpy as np

(Mostly from pennylane intro tutorials)
In this Pennylane intro, we will be optimizing two rotation matrices that flip a 0 state into a 1 state.
We will apply two rotation gates to our initial 0 state and measure the expectation value of the Pauli-Z operator. Why we use the Pauli-Z operator will become apparent later.

In [2]:
dev1 = qml.device("default.qubit", wires=1)
@qml.qnode(dev1)
def circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    return qml.expval(qml.PauliZ(0))

Now that we have our circuit set up, we need to define a loss function to minimize. 
Due to the way we have constructed our circuit, we can just use the output of our circuit as our loss function. We can do this because if our rotations successfully produce a 1 state, our output will be -1. If it produces a 0 state, our output will be 1. Minimizing loss, therefore, will yield a circuit that outputs -1 and therefore produces a 1 state.

We will use gradient descent in order to optimize our circuit.

In [3]:
dcircuit = qml.grad(circuit, argnum=0)

In [4]:
def cost(x):
    return circuit(x)

init_params = np.array([0.01, 0.01], requires_grad=True)

# initialise the optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.4)

# set the number of steps
steps = 100
# set the initial parameter values
params = init_params

for i in range(steps):
    # update the circuit parameters
    params = opt.step(cost, params)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))

print("Optimized rotation angles: {}".format(params))

Cost after step     5:  0.9961778
Cost after step    10:  0.8974944
Cost after step    15:  0.1440490
Cost after step    20: -0.1536720
Cost after step    25: -0.9152496
Cost after step    30: -0.9994046
Cost after step    35: -0.9999964
Cost after step    40: -1.0000000
Cost after step    45: -1.0000000
Cost after step    50: -1.0000000
Cost after step    55: -1.0000000
Cost after step    60: -1.0000000
Cost after step    65: -1.0000000
Cost after step    70: -1.0000000
Cost after step    75: -1.0000000
Cost after step    80: -1.0000000
Cost after step    85: -1.0000000
Cost after step    90: -1.0000000
Cost after step    95: -1.0000000
Cost after step   100: -1.0000000
Optimized rotation angles: [7.15266381e-18 3.14159265e+00]


For a 0.4 step size, this converges after about 40 steps. Let's deviate from the pennylane tutorial and explore which step size converges the fastest.

In [6]:
stepsizes = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
optimizers = [qml.GradientDescentOptimizer(stepsize=step) for step in stepsizes]

steps = 100

for optimizer in optimizers:
    params = init_params
    num = 0
    for i in range(steps):
        # update the circuit parameters
        num += 1
        params = optimizer.step(cost, params)

        if cost(params) <= -1:
            print("Number of steps taken to completely converge: " + str(num))
            break
            

Number of steps taken to completely converge: 81
Number of steps taken to completely converge: 59
Number of steps taken to completely converge: 46
Number of steps taken to completely converge: 37
Number of steps taken to completely converge: 30
Number of steps taken to completely converge: 25
Number of steps taken to completely converge: 21


Looks like in this case, larger stepsizes cause our algorithm to converge faster. This is not always the case, though.